## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-31-02-27-00 +0000,wsj,"Stock Market Today: U.S., Korea Reach Trade De...",https://www.wsj.com/livecoverage/fed-meeting-i...
1,2025-07-31-02-22-10 +0000,nyt,Judge Bars Trump Administration From Punishing...,https://www.nytimes.com/2025/07/30/us/politics...
2,2025-07-31-02-19-10 +0000,nypost,7-month-old girl ‘spared’ in quadruple homicid...,https://nypost.com/2025/07/30/us-news/7-month-...
3,2025-07-31-02-09-32 +0000,startribune,"Lynx defeat Liberty, 100-93 in rematch of 2024...",https://www.startribune.com/lynx-defeat-libert...
4,2025-07-31-01-58-30 +0000,nyt,Gaza Hunger Presents Trump With Moral Test Fam...,https://www.nytimes.com/2025/07/30/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2325/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,52
281,tsunami,20
122,tariffs,16
134,new,13
72,tariff,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
68,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...,116
235,2025-07-30-14-05-03 +0000,nypost,Trump threatens India with 25% tariff — plus ‘...,https://nypost.com/2025/07/30/us-news/trump-th...,114
198,2025-07-30-16-33-22 +0000,bbc,Trump to hit India with 25% tariffs - plus 'pe...,https://www.bbc.com/news/articles/cz71w1dgl84o...,108
227,2025-07-30-14-54-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...,98
133,2025-07-30-20-33-00 +0000,wsj,U.S. Raises Brazil Tariffs and Sanctions Judge...,https://www.wsj.com/world/americas/u-s-sanctio...,95


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
68,116,2025-07-30-22-42-00 +0000,nypost,Trump announces South Korea trade deal with $3...,https://nypost.com/2025/07/30/us-news/trump-an...
141,89,2025-07-30-20-19-00 +0000,wsj,Former Vice President Kamala Harris said Wedne...,https://www.wsj.com/politics/elections/kamala-...
298,63,2025-07-30-04-38-58 +0000,bbc,Watch: Major earthquake off Russia triggers wi...,https://www.bbc.com/news/videos/cwy01l5kgxno?a...
22,57,2025-07-31-00-53-35 +0000,nypost,NYC shooting victim Julia Hyman remembered as ...,https://nypost.com/2025/07/30/us-news/nyc-shoo...
239,56,2025-07-30-13-51-11 +0000,cbc,Bank of Canada holds interest rate at 2.75% as...,https://www.cbc.ca/news/business/boc-july-deci...
152,42,2025-07-30-19-43-47 +0000,nypost,Texas Republicans pitch new House map to net t...,https://nypost.com/2025/07/30/us-news/texas-re...
179,41,2025-07-30-18-13-51 +0000,nypost,Fed keeps rates unchanged — but two vote again...,https://nypost.com/2025/07/30/business/fed-kee...
208,40,2025-07-30-16-13-23 +0000,nyt,"Pro-Palestinian Group Can Appeal U.K. Ban, Jud...",https://www.nytimes.com/2025/07/30/world/europ...
158,38,2025-07-30-19-20-30 +0000,nypost,Israel suggests it could annex parts of Gaza i...,https://nypost.com/2025/07/30/world-news/israe...
230,37,2025-07-30-14-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
